In [4]:
!pip install mediapipe opencv-python

##**Step-1:** Import Libraries

In [20]:
#import modules
import cv2
import mediapipe as mp
import numpy as np
import time
from google.colab.patches import cv2_imshow
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# Pose Landmark Model
<img src= 'https://google.github.io/mediapipe/images/mobile/pose_tracking_full_body_landmarks.png'>

In [23]:
#list of all the given landmarks in mediapipe
for lndmrk in mp_pose.PoseLandmark:
    print(lndmrk)

PoseLandmark.NOSE
PoseLandmark.LEFT_EYE_INNER
PoseLandmark.LEFT_EYE
PoseLandmark.LEFT_EYE_OUTER
PoseLandmark.RIGHT_EYE_INNER
PoseLandmark.RIGHT_EYE
PoseLandmark.RIGHT_EYE_OUTER
PoseLandmark.LEFT_EAR
PoseLandmark.RIGHT_EAR
PoseLandmark.MOUTH_LEFT
PoseLandmark.MOUTH_RIGHT
PoseLandmark.LEFT_SHOULDER
PoseLandmark.RIGHT_SHOULDER
PoseLandmark.LEFT_ELBOW
PoseLandmark.RIGHT_ELBOW
PoseLandmark.LEFT_WRIST
PoseLandmark.RIGHT_WRIST
PoseLandmark.LEFT_PINKY
PoseLandmark.RIGHT_PINKY
PoseLandmark.LEFT_INDEX
PoseLandmark.RIGHT_INDEX
PoseLandmark.LEFT_THUMB
PoseLandmark.RIGHT_THUMB
PoseLandmark.LEFT_HIP
PoseLandmark.RIGHT_HIP
PoseLandmark.LEFT_KNEE
PoseLandmark.RIGHT_KNEE
PoseLandmark.LEFT_ANKLE
PoseLandmark.RIGHT_ANKLE
PoseLandmark.LEFT_HEEL
PoseLandmark.RIGHT_HEEL
PoseLandmark.LEFT_FOOT_INDEX
PoseLandmark.RIGHT_FOOT_INDEX


# Run Main Cell

In [27]:
#Setting up OpenCV and variables required
cap = cv2.VideoCapture('KneeBend.mp4')
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
fourcc = cv2.VideoWriter_fourcc(*'MP4V')
result = cv2.VideoWriter('final_output.mp4', fourcc, 20.0, (frame_width,frame_height))

import time
# Curl counter variables
counter = 0 
stage = None
toggle=0
togglesec=0
togglesecfin=0
feedback=0

#Calculating the angle
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    if angle >180.0:
        angle = 360-angle
    return angle 

framecount=0
secs=0

#------------------------------------------------------------------------------------------------

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    
    #Main while loop
    while cap.isOpened():
        ret, frame = cap.read()
        if(ret):
            framecount+=1
            if(framecount%25==0):
                secs+=1
            '''fps = cap.get(cv2.cv2.CAP_PROP_FPS)
            print("fps",fps) # finding fps... observed of video 25 frames/sec'''
            # Recolor image to RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
            # Make detection
            results = pose.process(image)
            # Recolor back to BGR
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            # Extract landmarks
            try:
                landmarks = results.pose_landmarks.landmark
                # Get coordinates
                hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]#23
                knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]#25
                ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]#27
                #print(knee)
                # Calculate angle
                angle = calculate_angle(hip, knee, ankle)
                # Visualize angle
                cv2.putText(image, str(angle), 
                               tuple(np.multiply(knee, [640, 480]).astype(int)), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                    )
                
                # Curl counter logic
                if angle > 160 and toggle==1:
                    stage = "down"
                    toggle=0
                    print("Knee down for",togglesecfin," seconds")
                    if(togglesecfin<8):
                        feedback=1
                    togglesecfin=0
                    togglesec=0

                if angle > 160 and toggle==0:
                    stage = "down"

                if angle < 140 and stage =='down' and toggle==0:
                    feedback=0
                    toggle=1
                    togglesec=secs
                    stage="up"
                    counter +=1
                    #print(counter)

                if angle < 140 and stage =='up' and toggle==1:
                    toggle=1
                    togglesecfin=secs-togglesec
                    #print(secs,togglesec)
                    #togglesec=togglesecfin
                    #print(togglesec)
                    stage="up"
                    #counter +=1
                    #print(counter)

            except:
                pass

            # Render curl counter
            # Setup status box

            if(feedback==0):
                cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1)
                # Rep data
                cv2.putText(image, 'REPS', (15,12), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
                cv2.putText(image, str(counter), 
                            (10,60), 
                            cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)

                # Stage data
                cv2.putText(image, 'STAGE', (65,12), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
                cv2.putText(image, stage, 
                            (80,60), 
                            cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
                cv2.putText(image, str(secs), 
                            (450,60), 
                            cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
                # Render detections
                mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                        mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                        mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                         )               
                cv2.imshow('Mediapipe Feed', image)
                result.write(image)
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
            else:
                cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1)
                # Rep data
                cv2.putText(image, 'REPS', (15,12), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
                cv2.putText(image, str(counter), 
                            (10,60), 
                            cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)

                # Stage data
                cv2.putText(image, 'STAGE', (65,12), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
                cv2.putText(image, stage, 
                            (80,60), 
                            cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
                cv2.putText(image, 'KEEP FEET BENT FOR MORE THAN 8 SECONDS!', (150,150), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 1, cv2.LINE_AA)
                cv2.putText(image, str(secs), 
                            (450,60), 
                            cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
                # Render detections
                mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                        mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                        mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                         )               
                cv2.imshow('Mediapipe Feed', image)
                result.write(image)
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break

    cap.release()
    cv2.destroyAllWindows()
    result.release()